# Recommendation - Model 🍿

---

<img src="https://visithrastnik.si/uploads/tic/public/generic_list_item/6-kulturna_prireditev_v_avli_kulturnega_centra_zagorje_ob_savi.jpg" />

---

In [9]:
!pip install lightfm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=867254 sha256=14bd47e4bf25c1b9dda241f156872f9411e4d70a494c43473828cd77ef924520
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


Now, time for the exciting part! We will train a Machine Learning model based on our previous **ratings** sparse matrix, so that it creates a recommendation engine automatically! 

First, load again the dataframe `movies` and `ratings`

In [6]:
### TODO: load the movies and ratings datasets
import pandas as pd
movies=pd.read_csv("ml-latest-small/movies.csv")
ratings=pd.read_csv("ml-latest-small/ratings.csv")
print(movies.head())
print(ratings.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


**Q1**. Start by loading all the pickle you saved during last challenge: `ratings_matrix`, `idx_to_mid`, `mid_to_idx`, `uid_to_idx`, `idx_to_uid`

In [7]:
import pickle
ratings_matrix=pickle.load(open('data/ratings_matrix.pkl','rb'))
idx_to_mid=pickle.load(open('data/idx_to_mid.pkl','rb'))
mid_to_idx=pickle.load(open('data/mid_to_idx.pkl','rb'))
uid_to_idx=pickle.load(open('data/uid_to_idx.pkl','rb'))
idx_to_uid=pickle.load(open('data/idx_to_uid.pkl','rb'))

**Q2**. Because the dataset is slightly different from what we have been used to (X as features, y as target), the usual `train_test_split` method from scikit-learn does not apply.

Hopefully, `lightfm` comes with a `random_train_test_split` located into `cross_validation` dedicated to this usecase 🙂

Split the data randomly into a `train` matrix and a `test` matrix with 20% of interactions into the test set.

In [11]:
import numpy as np
from lightfm.cross_validation import random_train_test_split
train,test=random_train_test_split(ratings_matrix,test_percentage=0.2)

**Q3**. Train a LightFM model for 10 epochs. You can use the parameter `loss="warp"`.

In [12]:
from lightfm import LightFM
model=LightFM(no_components=100,loss="warp",random_state=0)
model.fit(train,epochs=10, verbose=True)


Epoch: 100%|██████████| 10/10 [00:02<00:00,  3.49it/s]


**Q4**. Evaluate your model on your test set. You can use the `precision_at_k` metric implemented in the LightFM library.

In [13]:
from lightfm.evaluation import precision_at_k
k=5
precision_k=precision_at_k(model, test, train, k=k).mean()
print("precision at",k,"is:",precision_k)


precision at 5 is: 0.27783254


**Q5**. What does the attribute `item_embeddings` of `model` contains?  This will be the heart of your recommendation engine! 💟 So make sure you understand fully what it contains.

In [14]:
print(model.item_embeddings.shape)

(3650, 100)


In [15]:
It contians the number of movies(3650) and the number of components (100)

SyntaxError: ignored

**Q6**. We just trained a model that factorized our ratings matrix into a U matrix of shape (n_users, no_components) : `model.user_embeddings` ; and V matrix of shape (n_movies, no_components) : `model.item_embeddings`).

Now we want to compute **similarity between each pair of movies**.

> 🔦 **Hint**: For the similarity distance we can either use `cosine_similarity` function or `pearson_similarity`:
> - **Cosine similarity** between two vectors, or matrices X and Y is given by:
> ``` python
> from sklearn.metrics.pairwise import cosine_similarity
> cosine_similarity(X, Y)
> ```
> - **Pearson similarity** between two vectors, or matrices X and Y is given by:
> ``` python
> import numpy as np
> np.corrcoef(X, Y)
> ```

Compute the `similarity_scores` of size (n_movies, n_movies), containing for each element (i, j) the similarity between movie of index i and movie of index j.

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores=cosine_similarity(model.item_embeddings)
print(similarity_scores.shape)
similarity_scores

(3650, 3650)


array([[ 1.        ,  0.46697652,  0.21974115, ..., -0.3195277 ,
        -0.09163348, -0.22818698],
       [ 0.46697652,  1.0000001 ,  0.2787885 , ..., -0.40869108,
        -0.2217182 ,  0.03891685],
       [ 0.21974115,  0.2787885 ,  1.0000001 , ..., -0.37936372,
        -0.4204208 ,  0.0308933 ],
       ...,
       [-0.3195277 , -0.40869108, -0.37936372, ...,  0.99999994,
         0.6500985 , -0.13810055],
       [-0.09163348, -0.2217182 , -0.4204208 , ...,  0.6500985 ,
         1.0000001 , -0.35016114],
       [-0.22818698,  0.03891685,  0.0308933 , ..., -0.13810055,
        -0.35016114,  1.        ]], dtype=float32)

**Q7**. For movie of idx 20, what are the idx of the 10 most similar movies?

In [17]:
idx=20
similarity_idx=similarity_scores[idx]
ranked_idx=np.argsort(-similarity_idx)
ranked_mid=[idx_to_mid[x] for x in ranked_idx]
for mid in ranked_mid [:10]:
  print(movies[movies.movieId==mid]['title'])

314    Forrest Gump (1994)
Name: title, dtype: object
277    Shawshank Redemption, The (1994)
Name: title, dtype: object
945    Dead Poets Society (1989)
Name: title, dtype: object
322    Lion King, The (1994)
Name: title, dtype: object
418    Jurassic Park (1993)
Name: title, dtype: object
257    Pulp Fiction (1994)
Name: title, dtype: object
2078    Sixth Sense, The (1999)
Name: title, dtype: object
325    Mask, The (1994)
Name: title, dtype: object
1291    Titanic (1997)
Name: title, dtype: object
1939    Matrix, The (1999)
Name: title, dtype: object


**Q8**. Let's now test our engine! Suppose we have an user that likes **Toy Story** 🧸 (movie_id = 1). Which movies would you recommend to that user? In other words, which movies are the most similar to the movie Toy Story 

> ⚠️ **Warning**: Remember that your `similarity_scores` works with `idx` and you have the `movie_id` associated to your movie.

Retrieve the **top 5 recommendations**.

In [18]:
idx=mid_to_idx[1]
similarity_idx=similarity_scores[idx]
ranked_idx=np.argsort(-similarity_idx)
ranked_mid=[idx_to_mid[x] for x in ranked_idx]
for mid in ranked_mid [:5]:
  print(movies[movies.movieId==mid]['title'])

0    Toy Story (1995)
Name: title, dtype: object
615    Independence Day (a.k.a. ID4) (1996)
Name: title, dtype: object
418    Jurassic Park (1993)
Name: title, dtype: object
314    Forrest Gump (1994)
Name: title, dtype: object
322    Lion King, The (1994)
Name: title, dtype: object


As the next step is to **deploy your model**, you need now to: 

**Q9**. Save your `similarity_scores` into pickle format. Save also `movies` DataFrame into pickle format. Save them at the `data/netflix` directory at the root of the repository.

In [19]:
directory="./data"
pickle.dump(similarity_scores, open(directory+"/similarity_scores.pkl","wb"))
pickle.dump(movies, open(directory+"/movies.pkl","wb"))

**Q10**. Encapsulate the previous code into functions, especially you will need:
- `get_sim_scores(mid)` function that returns the vector of the similarity scores `sims` between a movie `mid` and all the other movies
- `get_ranked_recos(sims)` that returns for a vector of similarity scores `sims` the list of all ranked recommendations (n_movies) (from most recommended to least recommended) - in the format list of (mid, score, name) tuple.

In [28]:
def get_movie_name(mid, movies):
  try:
    name=movies.loc[movies.movieId==mid].title.values[0]
  except:
    name='Unknown'
  return name

def get_sims_scores(mid):
  idx=mid_to_idx[mid]
  sims=similarity_scores[idx]
  return sims
def get_ranked_recos(sims,movies):
  recos=[]
  for idx in np.argsort(-sims):
    mid=idx_to_mid[idx]
    name=get_movie_name(mid,movies)
    scores=sims[idx]
    recos.append((mid, scores,name))
  return recos
def get_recommendation(mid, movies,k):
  sim_scores=get_sims_scores(mid)
  return get_ranked_recos(sim_scores,movies)[:k]



In [29]:
get_recommendation(3,movies,10)

[(3, 1.0000001, 'Grumpier Old Men (1995)'),
 (880, 0.7566115, 'Island of Dr. Moreau, The (1996)'),
 (788, 0.6526669, 'Nutty Professor, The (1996)'),
 (141, 0.63321775, 'Birdcage, The (1996)'),
 (784, 0.6121417, 'Cable Guy, The (1996)'),
 (410, 0.6117396, 'Addams Family Values (1993)'),
 (339, 0.6050837, 'While You Were Sleeping (1995)'),
 (628, 0.60255796, 'Primal Fear (1996)'),
 (553, 0.60216796, 'Tombstone (1993)'),
 (432, 0.59364045, "City Slickers II: The Legend of Curly's Gold (1994)")]

If you have extra time, feel free now to improve your recommendation engine!